# generate state farm sample

first lets make sure we are in the right working directory

In [3]:
import os
os.getcwd()

'/home/ubuntu/nbs/brencklebox/data/statefarm'

## validation set

first we need to make a validation folder in this folder

In [4]:
os.mkdir('valid')

now we need to make a folder for each class within the validation folder

In [8]:
os.mkdir('valid/c0')
os.mkdir('valid/c1')
os.mkdir('valid/c2')
os.mkdir('valid/c3')
os.mkdir('valid/c4')
os.mkdir('valid/c5')
os.mkdir('valid/c6')
os.mkdir('valid/c7')
os.mkdir('valid/c8')
os.mkdir('valid/c9')

now we need to find out which images from each class belong to each driver

In [21]:
import pandas as pd

df = pd.read_csv('driver_imgs_list.csv')
grouped = df.groupby(['subject', 'classname']).agg(lambda x: tuple(x)).reset_index()
grouped['num_imgs'] = grouped.img.apply(len)
grouped.head(5)

,subject,classname,img,num_imgs
0,p002,c0,"(img_44733.jpg, img_72999.jpg, img_25094.jpg, ...",76
1,p002,c1,"(img_16856.jpg, img_8410.jpg, img_15559.jpg, i...",74
2,p002,c2,"(img_42941.jpg, img_22889.jpg, img_101258.jpg,...",86
3,p002,c3,"(img_46464.jpg, img_1565.jpg, img_44764.jpg, i...",79
4,p002,c4,"(img_48852.jpg, img_47934.jpg, img_40600.jpg, ...",84


so we can see that there aren't an equal number of images per subject/class lets get the total number of subjects

In [20]:
num_subjects = len(grouped.subject.unique())
print "there are", num_subjects, "subjects"

there are 26 subjects


lets figure out how many total images to put in our validation set

In [26]:
num_train_imgs = len(os.listdir('train/c0')) + \
                 len(os.listdir('train/c1')) + \
                 len(os.listdir('train/c2')) + \
                 len(os.listdir('train/c3')) + \
                 len(os.listdir('train/c4')) + \
                 len(os.listdir('train/c5')) + \
                 len(os.listdir('train/c6')) + \
                 len(os.listdir('train/c7')) + \
                 len(os.listdir('train/c8')) + \
                 len(os.listdir('train/c9'))

validation_set_proportion = .15                    
num_valid_imgs = int(validation_set_proportion * num_train_imgs)


print "there are", num_train_imgs, "training images"
print "therefore, we should have", num_valid_imgs, "validation images"

there are 22424 training images
therefore, we should have 3363 validation images


but if the number of images per class aren't proportional, we shouldn't sample them proportionally, so lets make sure we sample by the proportion in each class.

In [56]:
import random

def sample_df_tuple(tup, num):
    return random.sample(tup, int(num))

def sample_class (classname, valid_pct):    
    class_df = grouped[grouped.classname == classname].copy(deep=True)
    num_class_imgs = class_df.num_imgs.sum()
    valid = int(valid_pct * num_class_imgs)
    class_df['prop_imgs'] = class_df.num_imgs / num_class_imgs
    class_df['num_to_sample'] = (valid * class_df['prop_imgs']).round(0)
    class_df['samples'] = class_df.apply(lambda x: sample_df_tuple(x['img'], x['num_to_sample']), axis=1)
    samples = class_df['samples'].sum()
    return samples

class_names = ['c0', 'c1','c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
samples = []
for name in class_names:
    samples.extend(sample_class(name, 0.15))

print len(samples)
print num_valid_imgs

3349
3363


now we need to sample from each driver/class combination proportionally into the appropriate validation subfolder

## sample set

first we need to make a sample folder in this folder

next we need to make a train and a test folder within the sample folder

next we need to make a folder for each class within each of the test and validation folders

next we need to sample from each driver/class combination equally into the appropriate sample folder